In [ ]:
import graspy
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import truncnorm
from scipy.optimize import fmin_slsqp
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

from twins import load_dataset
%matplotlib inline

In [5]:
graphs = load_dataset(modality='fmri', parcellation='desikan_res-2x2x2', preprocess=None, ptr=None)[0]

df = pd.read_csv('../../../../twins/data/raw/unrestricted_jaewonc78_1_20_2019_23_7_58.csv')

In [13]:
gender = []
for sub in graphs.keys():
    gender.append(df[df.Subject == int(sub)]['Gender'].values[0])
    
le = LabelEncoder()
labels = le.fit_transform(gender)
# 0 is female, 1 is male

In [20]:
male_graphs = []
female_graphs = []

for idx, (sub, graph) in enumerate(graphs.items()):
    if labels[idx] == 1:
        male_graphs.append(graph)
    else:
        female_graphs.append(graph)
        
male_graphs = np.array(male_graphs)
female_graphs = np.array(female_graphs)

male_graphs_mean = male_graphs.mean(axis=0)
female_graphs_mean = female_graphs.mean(axis=0)

## Estiamate mean and variance of truncnorm for each edge

In [42]:
def estimate_params(data):
    def func(p, r, xa, xb):
        return truncnorm.nnlf(p, r)

    def constraint(p, r, xa, xb):
        a, b, loc, scale = p
        return np.array([a*scale + loc - xa, b*scale + loc - xb])

    xa, xb = 0, 1

    loc_guess = 0
    scale_guess = 1
    a_guess = (xa - loc_guess)/scale_guess
    b_guess = (xb - loc_guess)/scale_guess
    p0 = [a_guess, b_guess, loc_guess, scale_guess]

    a, b, mean, std = fmin_slsqp(func, p0, f_eqcons=constraint, args=(male_graphs[:, 0, 1], xa, xb),
                     iprint=False, iter=1000)
    
    return mean, std

In [48]:
verts = male_graphs.shape[-1]
male_means, male_vars = np.zeros((2, verts, verts))
female_means, female_vars = np.zeros((2, verts, verts))

for i in tqdm(range(verts)):
    for j in range(i + 1, verts):
        m, s = estimate_params(male_graphs[:, i, j])
        male_means[i, j] = m
        male_vars[i, j] = s
        
        m, s = estimate_params(female_graphs[:, i, j])
        female_means[i, j] = m
        female_vars[i, j] = s

/Users/j1c/miniconda3/envs/graspy/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


KeyboardInterrupt: 

In [46]:
male_means.shape

(70, 70)